# Help find a session
It is creating a sesisonId variable starting with a prefix

In [23]:
import os

def find_folder(start_with, path):
    for dir in os.listdir(path):
        if os.path.isdir(os.path.join(path, dir)) and dir.startswith(start_with):
            yield dir

start_with = "90"
s_path = "../sessions"

res = list(find_folder(start_with, s_path))
if len(res) == 1:
    sessionId = res[0]
    print(sessionId)
else:
    print('There are several (or none) sessions starting with the prefix:', start_with)
    print(res)

9092e9de-db04-45ad-bcc8-1d3f71bfeac1


# Read Context
In the current session open and display the context.pickle file

In [18]:
import pickle

with open(os.path.join(s_path, sessionId, 'context.pickle'), 'rb') as pfile:
    context = pickle.load(pfile)
print(context)

{'sessionId': '49719b7f-c9a0-4bf8-95e9-c867283748d2', 'domainRoot': 'E:\\Work\\A5\\production_architecture\\luigi', 'taskSource': 'validations', 'taskName': 'ValidationsTask', 'userId': 'marius', 'taskParams': {'inputfile': 'test_1000_lines.csv'}}


# Run status for a session
A good question is if this must be allowed to be!

In [ ]:
import luigi
import importlib
import luigi.tools.deps_tree as deps_tree
import sys

# tools.status_luigi(context, context['sessionId'])
config = luigi.configuration.LuigiConfigParser.instance()
config.set('session', 'id', sessionId)
sys.path.insert(0, '..')
module = importlib.import_module(f'tasks.{context["taskSource"]}')
task_class = getattr(module, context['taskName'])
task_obj = task_class(**context['taskParams'])
print('\n', deps_tree.print_tree(task_obj), '\n')


# Get from staged sqlite file all outputs

In [24]:
import sqlite3

# Connect to the SQLite database
with sqlite3.connect(os.path.join(s_path, sessionId, 'stage.sqlite')) as conn:
    # Create a cursor object to execute SQL commands
    cur =conn.cursor()
    # Execute a SELECT statement to fetch all rows from the table
    cur.execute("SELECT * FROM targets")
    # Fetch all rows and print them
    rows = cur.fetchall()
    if len(rows): 
        for row in rows:
            print(row)
    else:
        print('nothing to print')
    cur.close()

(1, 'Validation3(inputfile=test_1000_lines.csv)')
(2, 'Validation1(inputfile=test_1000_lines.csv)')


In [25]:
# get all tables
# Connect to the SQLite database
with sqlite3.connect(os.path.join(s_path, sessionId, 'stage.sqlite')) as conn:
    # Create a cursor object to execute SQL commands
    cur = conn.cursor()

    # Execute a SELECT statement to fetch the names of all tables
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")

    # Fetch all table names and print them
    tables = cur.fetchall()
    for table in tables:
        print(table[0])

    # Close the cursor and database connection
    cur.close()


targets
